# https://www.kaggle.com/c/digit-recognizer/
# with Tensorflow, the CPU mode took too much time on KNN.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
import tensorflow as tf
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
config = tf.ConfigProto(device_count={"CPU": 12}, # limit to num_cpu_core CPU usage
                inter_op_parallelism_threads = 1, 
                intra_op_parallelism_threads = 4,
                log_device_placement=True)

In [3]:
# load csv files to numpy arrays

def load_data():
    train = pd.read_csv("train.csv")
    print(train.shape)
    X_train = train.values[:,1:] 
    y_train = train.values[:,0]
    
    
    Pred_test = pd.read_csv("test.csv").values
    return X_train, y_train, Pred_test

x_train, y_train, pred_test = load_data()

(42000, 785)


In [4]:
print(x_train.shape, y_train.shape, pred_test.shape)

(42000, 784) (42000,) (28000, 784)


In [5]:
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=0.8, random_state=0)
#print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [6]:
'''pca = PCA(n_components=50)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)'''

'pca = PCA(n_components=50)\npca.fit(x_train)\nx_train = pca.transform(x_train)\nx_test = pca.transform(x_test)'

In [7]:
sess = tf.InteractiveSession(config=config)

In [8]:
'''
top_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
#def get_prediction(distance, k):
# Get k-min distance index (Nearest neighbor)
top_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
prediction_indices = tf.gather(y_train, top_k_indices)

y, idx, cnts = tf.unique_with_counts(prediction_indices,)
max_index = tf.argmax(cnts) #vote it from k

# Predict the mode category
#    return y[max_index]
'''

'\ntop_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)\n#def get_prediction(distance, k):\n# Get k-min distance index (Nearest neighbor)\ntop_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)\nprediction_indices = tf.gather(y_train, top_k_indices)\n\ny, idx, cnts = tf.unique_with_counts(prediction_indices,)\nmax_index = tf.argmax(cnts) #vote it from k\n\n# Predict the mode category\n#    return y[max_index]\n'

In [9]:
%%time
k = 5
batch_size=100

# Placeholders
x_data_train = tf.placeholder(shape=x_train.shape, dtype=tf.float32)
x_data_test = tf.placeholder(shape=[None, 784], dtype=tf.float32)
y_target_train = tf.placeholder(shape=y_train.shape, dtype=tf.float32)
y_target_test = tf.placeholder(shape=[batch_size], dtype=tf.float32)

distance = tf.reduce_sum(
    tf.abs(
        tf.subtract(x_data_train, tf.expand_dims(x_data_test, 1))
    ), axis=2)

# Prediction: Get min distance index (Nearest neighbor)
pred = tf.argmin(distance, 1)

prediction = tf.gather(y_target_train, pred)
#correct_prediction = tf.equal(prediction, y_target_test)
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Calculate how many loops over training data
num_loops = int(np.ceil(len(pred_test)/batch_size))

test_output = []
actual_vals = []
for i in range(num_loops):
    min_index = i*batch_size
    max_index = min((i+1)*batch_size,len(x_train))
    print(max_index)
    
    x_batch = pred_test[min_index:max_index]
    #x_batch = x_test[min_index:max_index]
    #y_batch = y_test[min_index:max_index]
    predictions = sess.run(prediction, feed_dict={x_data_train: x_train, 
                                                  x_data_test: x_batch,
                                                  y_target_train: y_train
                                                  #y_target_test: y_batch
                                                 })
    test_output.extend(predictions)
    #actual_vals.extend(y_batch)




#correct_prediction = tf.equal(test_output, actual_vals)
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy on test set: ' + str(accuracy))

predictions = np.array(test_output).astype(int)

def flush_predictons(predictions):
    imgs = pd.Series(range(1,pred_test.shape[0]+1))
    imgd = pd.DataFrame(imgs)
    
    labeld = pd.DataFrame(predictions)
    out = pd.concat([imgd, labeld],axis=1)
    out.columns = ["ImageId", "Label"]
    out.to_csv("KNN_PCA_Prediction.csv", index = False)

flush_predictons(predictions);

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

### Wall time: 1h 8min 57s